In [1]:
import requests
import pandas as pd
import datetime as dt
from fmiopendata.wfs import download_stored_query

# Retrieve the last 10 days daily observations + todays latest 10h observation
end_time = dt.datetime.utcnow()
start_time = end_time - dt.timedelta(days=10)
start_time2 = end_time - dt.timedelta(hours=10)
# Convert times to properly formatted strings
start_time = start_time.isoformat(timespec="seconds") + "Z"
start_time2 = start_time2.isoformat(timespec="seconds") + "Z"
# -> 2020-07-07T12:00:00Z
end_time = end_time.isoformat(timespec="seconds") + "Z"
# -> 2020-07-07T13:00:00Z

# For last 10d we get daily values
obs = download_stored_query("fmi::observations::weather::daily::multipointcoverage",
                            args=["place=kumpula",
                                  "starttime=" + start_time,
                                  "endtime=" + end_time])

# For last 10h we get all observations and use these to calculate the "daily" observations for today
obs2 = download_stored_query("fmi::observations::weather::multipointcoverage",
                            args=["place=kumpula",
                                  "starttime=" + start_time2,
                                  "endtime=" + end_time])

df = pd.DataFrame.from_dict({(i): obs.data[i][j]
                           for i in obs.data.keys() 
                           for j in obs.data[i].keys()},
                       orient='index')
df = df.applymap(lambda x: x.get('value'))

dfh = pd.DataFrame.from_dict({(i): obs2.data[i][j]
                           for i in obs2.data.keys() 
                           for j in obs2.data[i].keys()},
                       orient='index')
dfh = dfh.applymap(lambda x: x.get('value'))

# We have to calculate some new columns for these more frequent observations
dfh['Minimum temperature'] = dfh['Air temperature'].min()
dfh['Maximum temperature'] = dfh['Air temperature'].max()
dfh['Air temperature'] = dfh['Air temperature'].mean()
dfh['Ground minimum temperature'] = dfh['Minimum temperature']
dfh = dfh.sort_index(axis=0, ascending=False).head(1)
dfh = dfh[df.columns]

#print(df)

In [2]:
import importlib
import data.weather_data.weather_data_trimming as f
importlib.reload(f)

df2 = f.rename_cols(df)
dfh2 = f.rename_cols(dfh)
df2 = f.merge_and_replace(df2, dfh2)

#print(df2)

ModuleNotFoundError: No module named 'data.weather_data'

In [ ]:
import model.weather_feature_extraction as m
importlib.reload(m)

df3 = m.get_features(df2).sort_index(axis=0, ascending=False)
X_pred = df3.head(1).fillna(0).to_numpy() #
date = df3.head(1).index.item() + dt.timedelta(days=1)

#print(X_pred)

In [ ]:
import joblib

model = joblib.load('/slipskip/model/random_forest_model')

y_pred = model.predict(X_pred)
y_prob = model.predict_proba(X_pred)
print(date, y_pred, y_prob)